<a href="https://colab.research.google.com/github/IshanKapadia-Data/Instacart-Market-Basket-Analysis/blob/main/Market_Basket_Analysis_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

aisles       = pd.read_csv('aisles.csv')
departments  = pd.read_csv('departments.csv')
products     = pd.read_csv('products.csv')
orders       = pd.read_csv('orders.csv')
prior        = pd.read_csv('order_products__prior.csv')
train        = pd.read_csv('order_products__train.csv')

In [ ]:
prior_df = prior.merge(orders, on ='order_id', how='inner')
prior_df = prior_df.merge(products, on = 'product_id', how = 'left')
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13


In [ ]:
prior_df['user_buy_product_times'] = prior_df.groupby(['user_id', 'product_id']).cumcount() + 1
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [ ]:
agg_dict1 = {'add_to_cart_order' : {'mean_add_to_cart_order':'mean'},
           'reordered' : {'total_orders':'count', 'total_reorders':'sum', 'reorder_percentage':'mean'},
           'user_id': {'unique_users' :lambda x: x.nunique()},
           'user_buy_product_times': {'order_first_time_total_cnt' : lambda x: sum(x==1),
                                      'order_second_time_total_cnt' :lambda x: sum(x==2)},
           'product_name':{'is_organic': lambda x: 1 if 'Organic' in x else 0}}

In [ ]:
prod_feats1 = prior_df.groupby('product_id').agg(
    mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    total_orders=('reordered', 'count'),
    total_reorders=('reordered', 'sum'),
    reorder_percentage=('reordered', 'mean'),
    unique_users=('user_id', 'nunique'),
    order_first_time_total_cnt=('user_buy_product_times', lambda x: (x == 1).sum()),
    order_second_time_total_cnt=('user_buy_product_times', lambda x: (x == 2).sum()),
    is_organic=('product_name', lambda x: 1 if any('Organic' in str(i) for i in x) else 0)
).reset_index()
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic
0,1,5.801836,1852,1136,0.613391,716,716,276,0
1,2,9.888889,90,12,0.133333,78,78,8,0
2,3,6.415162,277,203,0.732852,74,74,36,0
3,4,9.507599,329,147,0.446809,182,182,64,0
4,5,6.466667,15,9,0.600000,6,6,4,0


In [ ]:
prod_feats1['second_time_percent'] = prod_feats1.order_second_time_total_cnt/prod_feats1.order_first_time_total_cnt

In [ ]:
agg_dict2 = {'add_to_cart_order' : {'aisle_mean_add_to_cart_order':'mean',
                                   'aisle_std_add_to_cart_order':'std'},
           'reordered' : {'aisle_total_orders':'count', 'aisle_total_reorders':'sum', 'aisle_reorder_percentage':'mean'},
           'user_id': {'aisle_unique_users' :lambda x: x.nunique()}}

In [ ]:
agg_dict2 = {'add_to_cart_order' : {'aisle_mean_add_to_cart_order':'mean',
                                   'aisle_std_add_to_cart_order':'std'},
           'reordered' : {'aisle_total_orders':'count', 'aisle_total_reorders':'sum', 'aisle_reorder_percentage':'mean'},
           'user_id': {'aisle_unique_users' :lambda x: x.nunique()}}

In [ ]:
aisle_feats = prior_df.groupby('aisle_id').agg(
    aisle_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    aisle_std_add_to_cart_order=('add_to_cart_order', 'std'),
    aisle_total_orders=('reordered', 'count'),
    aisle_total_reorders=('reordered', 'sum'),
    aisle_reorder_percentage=('reordered', 'mean'),
    aisle_unique_users=('user_id', 'nunique')
).reset_index()


In [ ]:
aisle_feats.head()

,aisle_id,aisle_mean_add_to_cart_order,aisle_std_add_to_cart_order,aisle_total_orders,aisle_total_reorders,aisle_reorder_percentage,aisle_unique_users
0,1,8.167640,7.104166,71928,42912,0.596597,20711
1,2,9.275497,7.473802,82491,40365,0.489326,31222
2,3,9.571935,7.899672,456386,272922,0.598007,63592
3,4,10.161450,7.745705,200687,98243,0.489533,53892
4,5,10.297600,8.187047,62510,17542,0.280627,32312


In [ ]:
agg_dict3 = {'add_to_cart_order' : {'department_mean_add_to_cart_order':'mean',
                                   'department_std_add_to_cart_order':'std'},
           'reordered' : {'department_total_orders':'count', 'department_total_reorders':'sum',
                          'department_reorder_percentage':'mean'},
           'user_id': {'department_unique_users' :lambda x: x.nunique()}}

In [ ]:
dpt_feats = prior_df.groupby('department_id').agg(
    department_mean_add_to_cart_order=('add_to_cart_order', 'mean'),
    department_std_add_to_cart_order=('add_to_cart_order', 'std'),
    department_total_orders=('reordered', 'count'),
    department_total_reorders=('reordered', 'sum'),
    department_reorder_percentage=('reordered', 'mean'),
    department_unique_users=('user_id', 'nunique')
).reset_index()


In [ ]:
dpt_feats.head()

,department_id,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users
0,1,8.996414,7.393502,2236432,1211890,0.541885,163233
1,2,8.277645,7.526272,36291,14806,0.407980,17875
2,3,8.084397,6.904849,1176787,739188,0.628141,140612
3,4,8.022875,6.658899,9479291,6160710,0.649913,193237
4,5,5.428346,5.778253,153696,87595,0.569924,15798


In [ ]:
prod_feats1 = prod_feats1.merge(products, on = 'product_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisle_feats, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(aisles, on = 'aisle_id', how = 'left')
prod_feats1 = prod_feats1.merge(dpt_feats, on = 'department_id', how = 'left')
prod_feats1 = prod_feats1.merge(departments, on = 'department_id', how = 'left')
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [ ]:
prod_feats1.drop(['product_name', 'aisle_id', 'department_id'], axis = 1, inplace = True)
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,aisle_reorder_percentage,aisle_unique_users,aisle,department_mean_add_to_cart_order,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,0.548698,54202,cookies cakes,9.187743,7.692492,2887550,1657973,0.574180,174219,snacks
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,0.152391,76402,spices seasonings,9.593425,7.875241,1875577,650301,0.346721,172755,pantry
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,0.527615,53197,tea,6.976699,6.711172,2690129,1757892,0.653460,172795,beverages
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,0.556655,58749,frozen meals,8.996414,7.393502,2236432,1211890,0.541885,163233,frozen
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,0.280627,32312,marinades meat preparation,9.593425,7.875241,1875577,650301,0.346721,172755,pantry


In [ ]:
prod_feats1.shape

(49677, 24)

In [ ]:
prod_feats1.dtypes

,0
product_id,int64
mean_add_to_cart_order,float64
total_orders,int64
total_reorders,int64
reorder_percentage,float64
unique_users,int64
order_first_time_total_cnt,int64
order_second_time_total_cnt,int64
is_organic,int64
second_time_percent,float64


In [ ]:
!pip install category_encoders
import category_encoders as ce
encoder= ce.BinaryEncoder(cols=['aisle', 'department'],return_df=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
prod_feats1 = encoder.fit_transform(prod_feats1)
prod_feats1.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,7.692492,2887550,1657973,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,6.711172,2690129,1757892,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,7.393502,2236432,1211890,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0


In [ ]:
prod_feats1.shape

(49677, 35)

In [ ]:
prod_feats1.columns

Index(['product_id', 'mean_add_to_cart_order', 'total_orders',
       'total_reorders', 'reorder_percentage', 'unique_users',
       'order_first_time_total_cnt', 'order_second_time_total_cnt',
       'is_organic', 'second_time_percent', 'aisle_mean_add_to_cart_order',
       'aisle_std_add_to_cart_order', 'aisle_total_orders',
       'aisle_total_reorders', 'aisle_reorder_percentage',
       'aisle_unique_users', 'aisle_0', 'aisle_1', 'aisle_2', 'aisle_3',
       'aisle_4', 'aisle_5', 'aisle_6', 'aisle_7',
       'department_mean_add_to_cart_order', 'department_std_add_to_cart_order',
       'department_total_orders', 'department_total_reorders',
       'department_reorder_percentage', 'department_unique_users',
       'department_0', 'department_1', 'department_2', 'department_3',
       'department_4'],
      dtype='object')

In [ ]:
prod_feats1.isnull().any().any()

np.False_

In [ ]:
prior_df.isnull().any()

,0
order_id,False
product_id,False
add_to_cart_order,False
reordered,False
user_id,False
eval_set,False
order_number,False
order_dow,False
order_hour_of_day,False
days_since_prior_order,True


In [ ]:
prior_df.days_since_prior_order = prior_df.days_since_prior_order.fillna(0)

In [ ]:
prior_df.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times
0,2,33120,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,1
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,1
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,1
3,2,45918,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,1
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,1


In [ ]:
agg_dict4 = {'order_dow': {'avg_dow':'mean', 'std_dow':'std'},
           'order_hour_of_day': {'avg_doh':'mean', 'std_doh':'std'},
           'days_since_prior_order': {'avg_since_order':'mean', 'std_since_order':'std'},
           'order_number': {'total_orders_by_user': lambda x: x.nunique()},
           'product_id': {'total_products_by_user': 'count',
                         'total_unique_product_by_user': lambda x: x.nunique()},
           'reordered': {'total_reorders_by_user':'sum',
                        'reorder_propotion_by_user':'mean'}}

In [ ]:
user_feats = prior_df.groupby('user_id').agg(
    avg_dow=('order_dow', 'mean'),
    std_dow=('order_dow', 'std'),
    avg_doh=('order_hour_of_day', 'mean'),
    std_doh=('order_hour_of_day', 'std'),
    avg_since_order=('days_since_prior_order', 'mean'),
    std_since_order=('days_since_prior_order', 'std'),
    total_orders_by_user=('order_number', 'nunique'),
    total_products_by_user=('product_id', 'count'),
    total_unique_product_by_user=('product_id', 'nunique'),
    total_reorders_by_user=('reordered', 'sum'),
    reorder_propotion_by_user=('reordered', 'mean')
).reset_index()

user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user
0,1,2.644068,1.256194,10.542373,3.500355,18.542373,10.559065,10,59,18,41,0.694915
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000
3,4,4.722222,0.826442,13.111111,1.745208,11.944444,9.973330,5,18,17,1,0.055556
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378


In [ ]:
user_feats2 = prior_df.groupby(['user_id', 'order_number']).agg(
    average_order_size=('reordered', 'count'),
    reorder_in_order=('reordered', 'mean')
).reset_index()


In [ ]:
user_feats2.head()

,user_id,order_number,average_order_size,reorder_in_order
0,1,1,5,0.000
1,1,2,6,0.500
2,1,3,5,0.600
3,1,4,5,1.000
4,1,5,8,0.625


In [ ]:
user_feats3 = user_feats2.groupby('user_id').agg({'average_order_size' : 'mean',
                                   'reorder_in_order':'mean'})
user_feats3 = user_feats3.reset_index()
user_feats3.head()

,user_id,average_order_size,reorder_in_order
0,1,5.900000,0.705833
1,2,13.928571,0.447961
2,3,7.333333,0.658817
3,4,3.600000,0.028571
4,5,9.250000,0.377778


In [ ]:
user_feats = user_feats.merge(user_feats3, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order
0,1,2.644068,1.256194,10.542373,3.500355,18.542373,10.559065,10,59,18,41,0.694915,5.900000,0.705833
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817
3,4,4.722222,0.826442,13.111111,1.745208,11.944444,9.973330,5,18,17,1,0.055556,3.600000,0.028571
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778


In [ ]:
last_three_orders = user_feats2.groupby('user_id')['order_number'].nlargest(3).reset_index()
last_three_orders.head()

,user_id,level_1,order_number
0,1,9,10
1,1,8,9
2,1,7,8
3,2,23,14
4,2,22,13


In [ ]:
last_three_orders = user_feats2.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1
0,1,8,6,0.666667,7
1,1,9,6,1.000000,8
2,1,10,9,0.666667,9
3,2,12,19,0.578947,21
4,2,13,9,0.000000,22


In [ ]:
last_three_orders['rank'] = last_three_orders.groupby("user_id")["order_number"].rank("dense", ascending=True)

In [ ]:
last_order_feats = last_three_orders.pivot_table(index = 'user_id', columns = ['rank'], \
                                                 values=['average_order_size', 'reorder_in_order']).\
                                                reset_index(drop = False)
last_order_feats.columns = ['user_id','orders_3', 'orders_2', 'orders_1', 'reorder_3', 'reorder_2', 'reorder_1']
last_order_feats.head()

,user_id,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,9.0,5.0,12.0,0.444444,0.4,0.666667


In [ ]:
user_feats = user_feats.merge(last_order_feats, on = 'user_id', how = 'left')
user_feats.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542373,10.559065,10,59,18,41,0.694915,5.900000,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944444,9.973330,5,18,17,1,0.055556,3.600000,0.028571,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778,9.0,5.0,12.0,0.444444,0.4,0.666667


In [ ]:
agg_dict6 = {'reordered': {'total_product_orders_by_user':'count',
                          'total_product_reorders_by_user':'sum',
                          'user_product_reorder_percentage': 'mean'},
            'add_to_cart_order': {'avg_add_to_cart_by_user':'mean'},
            'days_since_prior_order': {'avg_days_since_last_bought':'mean'},
            'order_number': {'last_ordered_in':'max'}}

In [ ]:
user_product_feats = prior_df.groupby(['user_id', 'product_id']).agg(
    total_product_orders_by_user=('reordered', 'count'),
    total_product_reorders_by_user=('reordered', 'sum'),
    user_product_reorder_percentage=('reordered', 'mean'),
    avg_add_to_cart_by_user=('add_to_cart_order', 'mean'),
    avg_days_since_last_bought=('days_since_prior_order', 'mean'),
    last_ordered_in=('order_number', 'max')
).reset_index()


In [ ]:
user_product_feats.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in
0,1,196,10,9,0.900000,1.400000,17.600000,10
1,1,10258,9,8,0.888889,3.333333,19.555556,10
2,1,10326,1,0,0.000000,5.000000,28.000000,5
3,1,12427,10,9,0.900000,3.300000,17.600000,10
4,1,13032,3,2,0.666667,6.333333,21.666667,10


In [ ]:
last_three_orders.head()

,user_id,order_number,average_order_size,reorder_in_order,level_1,rank
0,1,8,6,0.666667,7,1.0
1,1,9,6,1.000000,8,2.0
2,1,10,9,0.666667,9,3.0
3,2,12,19,0.578947,21,1.0
4,2,13,9,0.000000,22,2.0


In [ ]:
last_orders = prior_df.merge(last_three_orders, on = ['user_id', 'order_number'], how = 'inner')
last_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,user_buy_product_times,average_order_size,reorder_in_order,level_1,rank
0,7,34050,1,0,142903,prior,11,2,14,30.0,Orange Juice,31,7,1,2,0.000000,2231251,2.0
1,7,46802,2,0,142903,prior,11,2,14,30.0,Pineapple Chunks,116,1,1,2,0.000000,2231251,2.0
2,14,20392,1,1,18194,prior,49,3,15,3.0,Hair Bender Whole Bean Coffee,26,7,1,11,0.818182,282882,1.0
3,14,27845,2,1,18194,prior,49,3,15,3.0,Organic Whole Milk,84,16,1,11,0.818182,282882,1.0
4,14,162,3,1,18194,prior,49,3,15,3.0,Organic Mini Homestyle Waffles,52,1,1,11,0.818182,282882,1.0


In [ ]:
last_orders['rank'] = last_orders.groupby(['user_id', 'product_id'])['order_number'].rank("dense", ascending=True)

In [ ]:
product_purchase_history = last_orders.pivot_table(index = ['user_id', 'product_id'],\
                                                   columns='rank', values = 'reordered').reset_index()
product_purchase_history.columns = ['user_id', 'product_id', 'is_reorder_3', 'is_reorder_2', 'is_reorder_1']
product_purchase_history.fillna(0, inplace = True)
product_purchase_history.head()

,user_id,product_id,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,1.0,1.0,1.0
1,1,10258,1.0,1.0,1.0
2,1,12427,1.0,1.0,1.0
3,1,13032,1.0,0.0,0.0
4,1,25133,1.0,1.0,1.0


In [ ]:
user_product_feats = user_product_feats.merge(product_purchase_history, on=['user_id', 'product_id'], how = 'left')
user_product_feats.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.600000,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555556,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,NaN,NaN,NaN
3,1,12427,10,9,0.900000,3.300000,17.600000,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666667,10,1.0,0.0,0.0


In [ ]:
user_product_feats.isnull().sum()

,0
user_id,0
product_id,0
total_product_orders_by_user,0
total_product_reorders_by_user,0
user_product_reorder_percentage,0
avg_add_to_cart_by_user,0
avg_days_since_last_bought,0
last_ordered_in,0
is_reorder_3,8382738
is_reorder_2,8382738


In [ ]:
user_product_feats.fillna(0, inplace = True)

In [ ]:
# Define the root directory (this will save in your Colab working directory)
root = '/content/'

# Save your DataFrames as pickle files
prod_feats1.to_pickle(root + 'product_features.pkl')
user_feats.to_pickle(root + 'user_features.pkl')
user_product_feats.to_pickle(root + 'user_product_features.pkl')



In [ ]:
df = pd.read_pickle(root +'product_features.pkl')
df.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_std_add_to_cart_order,department_total_orders,department_total_reorders,department_reorder_percentage,department_unique_users,department_0,department_1,department_2,department_3,department_4
0,1,5.801836,1852,1136,0.613391,716,716,276,0,0.385475,...,7.692492,2887550,1657973,0.574180,174219,0,0,0,0,1
1,2,9.888889,90,12,0.133333,78,78,8,0,0.102564,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0
2,3,6.415162,277,203,0.732852,74,74,36,0,0.486486,...,6.711172,2690129,1757892,0.653460,172795,0,0,0,1,1
3,4,9.507599,329,147,0.446809,182,182,64,0,0.351648,...,7.393502,2236432,1211890,0.541885,163233,0,0,1,0,0
4,5,6.466667,15,9,0.600000,6,6,4,0,0.666667,...,7.875241,1875577,650301,0.346721,172755,0,0,0,1,0


In [ ]:
df = pd.read_pickle(root+'user_features.pkl')
df.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542373,10.559065,10,59,18,41,0.694915,5.900000,0.705833,6.0,6.0,9.0,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93,0.476923,13.928571,0.447961,19.0,9.0,16.0,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867396,12,88,33,55,0.625000,7.333333,0.658817,6.0,5.0,6.0,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944444,9.973330,5,18,17,1,0.055556,3.600000,0.028571,7.0,2.0,3.0,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14,0.378378,9.250000,0.377778,9.0,5.0,12.0,0.444444,0.4,0.666667


In [ ]:
df = pd.read_pickle(root + 'user_product_features.pkl')
df.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.600000,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555556,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,17.600000,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666667,10,1.0,0.0,0.0
